In [ ]:
import os
import sys
from pathlib import Path

# Navigate to project root (equivalent to cd ..)
project_dir = Path(__file__).parent.parent if '__file__' in globals() else Path.cwd().parent
os.chdir(project_dir)

# Add src directory to Python path for imports
src_dir = project_dir / "src"
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

# Set environment for dev testing
os.environ['REPORT_ENV'] = 'dev'

In [ ]:
"""
Upstream dependencies: Needs to run after silver account table (because that gets fed into create calculated columns). Portfolio key also needs
to run before this (that is an upstream of the silver account table).

Stucture for Customer Dimensional Modeling

Base Customer Dim is the first one created.

Feeds to Child tables: Pers + Org tables with specific data that can explicitly join to enrich data where needed. This is seamless for end user
in PowerBI or other method of data being served.

This is the building block to create a centralized view of the customer.
"""

import src.silver.customer_dim.fetch_data
import cdutils.add_effdate # type: ignore
import cdutils.deduplication # type: ignore
import cdutils.customer_dim # type: ignore
import cdutils.input_cleansing # type: ignore
import pandas as pd
import src.config
from deltalake import DeltaTable
import datetime

def sort_dedupe_raw(df, dedupe_list):
    if 'adddate' in df.columns:
        df_sorted = df.sort_values(by='adddate', ascending=False)
    else:
        pass

    result = cdutils.deduplication.dedupe(dedupe_list)
    return result

def calculate_age(row):
    dob = pd.to_datetime(row['datebirth'], errors='coerce')
    dod = pd.to_datetime(row['datebirth'], errors='coerce')

    if pd.isna(dob):
        return None

    if not pd.isna(dod):
        return -1

    effdate = pd.to_datetime(row.get('effdate', datetime.date.today()), errors='coerce')
    if pd.isna('effdate') or effdate <= dob:
        return None

    age_days = (effdate - dob).days
    age_years = age_days // 365.25

    # Cap at 125, assume deceased and we never updated database
    if age_years >= 125:
        return -1
    else:
        return int(age_years)

def generate_base_customer_dim_table():
    """
    Create Base Customer Dim table with P+persnbr or O+orgnbr as primary key
    """
    # data = src.silver.customer_dim.fetch_data.fetch_data()
    # wh_pers = data['wh_pers'].copy()
    # wh_org =data['wh_org'].copy()

    wh_pers = DeltaTable(src.config.BRONZE / "wh_pers").to_pandas()
    wh_org = DeltaTable(src.config.BRONZE / "wh_org").to_pandas()

    # Sort descending and dedupe primary keys 
    dedupe_list = [
            {'df':wh_org, 'field':'orgnbr'}
        ]
    wh_org = sort_dedupe_raw(wh_org, dedupe_list)

    dedupe_list = [
            {'df':wh_pers, 'field':'persnbr'}
        ]
    wh_pers = sort_dedupe_raw(wh_pers, dedupe_list)

    # Org + Persify
    wh_org = cdutils.customer_dim.orgify(wh_org, 'orgnbr')
    wh_pers = cdutils.customer_dim.persify(wh_pers, 'persnbr')

    # Add Customer type field
    wh_org['customer_type'] = 'Organization'
    wh_pers['customer_type'] = 'Person'

    # Concat fields
    customer_dim = pd.concat([wh_org, wh_pers], ignore_index=True)
    
    # Coalesce customer name + filter down
    customer_dim['customer_name'] = customer_dim['orgname'].fillna(customer_dim['persname'])
    customer_dim = customer_dim[[
        'customer_id',
        'customer_type',
        'customer_name',
        'adddate'
    ]].copy()

    # Add taxid
    data = src.silver.customer_dim.fetch_data.fetch_taxid_data()
    vieworgtaxid = data['vieworgtaxid'].copy()
    viewperstaxid = data['viewperstaxid'].copy()
    
    vieworgtaxid = cdutils.customer_dim.orgify(vieworgtaxid, 'orgnbr')
    viewperstaxid = cdutils.customer_dim.persify(viewperstaxid, 'persnbr')

    # Concat taxid
    taxid_concat = pd.concat([vieworgtaxid, viewperstaxid], ignore_index=True)
    assert taxid_concat['customer_id'].is_unique, "Duplicates on concat taxid"

    # Merge
    customer_dim = customer_dim.merge(taxid_concat, on='customer_id', how='left')

    # Join with account data to get counts and balances
    
    customer_dim = cdutils.add_effdate.add_effdate(customer_dim)

    return customer_dim

# def generate_pers_dim():
"""
Generate silver layer person dimension table
This table is a child of the base customer dim

Primary key is customer_id and will be used to enrich customer dimensional queries with info
that applies specifically to People, not organizations
"""
wh_pers = DeltaTable(src.config.BRONZE / "wh_pers").to_pandas()

# Sort descending and dedupe primary keys 
dedupe_list = [
        {'df':wh_pers, 'field':'persnbr'}
    ]
wh_pers = sort_dedupe_raw(wh_pers, dedupe_list)

# Persify
wh_pers = cdutils.customer_dim.persify(wh_pers, 'persnbr')

wh_pers = wh_pers[[
    'customer_id',
    'perssortname',
    'datebirth',
    'datedeath',
    'employeeyn',
    'privacyyn',
    'homeemail',
    'busemail'
]].copy()

wh_pers = cdutils.add_effdate.add_effdate(wh_pers)

wh_pers['age'] = wh_pers.apply(calculate_age, axis=1)

# Append first name and last name
data = src.silver.customer_dim.fetch_data.fetch_pers_data()
pers = data['pers'].copy()

# Sort descending and dedupe primary keys 
dedupe_list = [
        {'df':pers, 'field':'persnbr'}
    ]
pers = sort_dedupe_raw(pers, dedupe_list)

# Persify
pers = cdutils.customer_dim.persify(pers, 'persnbr')

pers = pers.rename(columns={
    'firstnameupper':'firstname',
    'lastnameupper':'lastname'
}).copy()

wh_pers = wh_pers.merge(pers, on='customer_id', how='left')

# return wh_pers


In [ ]:
import pandas as pd
import src.config
from deltalake import DeltaTable
import cdutils.customer_dim # type: ignore
import src.raynham_report.fetch_data
import cdutils.input_cleansing # type: ignore

def filter_distinct_customers_assigned_to_n_raynham(df):
    """
    Unique customers that are assigned to branch in question
    """
    filtered_df = df[df['branchname'] == 'BCSB - NORTH RAYNHAM BRANCH'].copy()
    distinct_customers = filtered_df[['customer_id']].drop_duplicates()
    distinct_customers['eligibility'] = 'Assigned to N Raynham'
    return distinct_customers

def filter_distinct_customers_transacted_at_n_raynham(accts):
    """
    Unique accounts that have transacted at branch in question
    """
    data = src.raynham_report.fetch_data.fetch_rtxn_data()
    transacted = data['transacted'].copy()
    transacted_schema = {
        'acctnbr':'str'
    }
    transacted = cdutils.input_cleansing.cast_columns(transacted, transacted_schema)
    transacted = transacted.merge(accts, how='inner', on='acctnbr')
    distinct_customers = transacted[['customer_id']].drop_duplicates()
    distinct_customers['eligibility'] = 'Transacted at N Raynham within 90 days'

    return distinct_customers 

def filter_distinct_customer_near_branches(accts):
    """
    Unique Customers located near raynham or taunton

    Requested was within 5 mi of branch, but for simplicity, we match on primary zip codes
    """
    zip_codes = ["02767","02780"]
    accts_schema = {
        'zipcd'
    }
    filtered_df = accts[accts['primaryownerzipcd'].isin(zip_codes)]
    distinct_customers = filtered_df[['customer_id']].drop_duplicates()
    distinct_customers['eligibility'] = 'Zip Code in Raynham (02767) or Taunton (02767)'
    return distinct_customers
    

# def generate_raynham_report():
"""
Objective:

To create a person mailing list to North raynham Branch Construction customers

Params:

Assigned to N Raynham
Transacted at N Raynham within 90 days
Address within a 5 mile radius of N. Raynham, Raynham Center, Main Office
Owns Safe Deposit Box @ N. Raynham
Exclusions (do we have standard exclusions for all mailings? Including common ones below)
Customers under 18 years of age
Deceased customers
Charged off accounts


Data fields:

First name
Last name
Address
City
State
Zip
"""

# Pull in Base Customer Layer
base_customer_dim = DeltaTable(src.config.SILVER / "base_customer_dim").to_pandas()

# Pull in Active Accounts (create a clean customer_id for joining)
accts = DeltaTable(src.config.SILVER / "account").to_pandas()
accts = cdutils.customer_dim.orgify(accts, 'taxrptfororgnbr')
accts = accts.rename(columns={'customer_id':'org_id'}).copy()
accts = cdutils.customer_dim.persify(accts, 'taxrptforpersnbr')
accts['customer_id'] = accts['customer_id'].fillna(accts['org_id'])
accts = accts.drop(columns=['org_id']).copy()

# Assigned to Branch from Accts

assigned = filter_distinct_customers_assigned_to_n_raynham(accts)

# Transacted at Branch in last 90 days
transacted = filter_distinct_customers_transacted_at_n_raynham(accts)

# Address near Branch (zip codes)
nearby = filter_distinct_customer_near_branches(accts)

# Union/concat eligible criteria on cust_id
concat_df = pd.concat([assigned, transacted, nearby], ignore_index=True)
concat_df = concat_df.drop_duplicates(subset=['customer_id'], keep='first')

# Inner join with base customer dim
base_customer_dim = base_customer_dim[[
    'customer_id',
    'customer_type',
    'customer_name'
]].copy()
customer_df = base_customer_dim.merge(concat_df, how='inner', on='customer_id')

# Append primary address
customer_address_link = DeltaTable(src.config.SILVER / "customer_address_link").to_pandas()
customer_address_link = customer_address_link[customer_address_link['addrusecd'] == 'PRI'].copy()
customer_address_link = customer_address_link[[
    'customer_id',
    'addrnbr'
]].copy()
customer_address_link_schema = {
    'addrnbr':'str'
}
customer_address_link = cdutils.input_cleansing.cast_columns(customer_address_link, customer_address_link_schema)

address = DeltaTable(src.config.SILVER / "address").to_pandas()
address_schema = {
    'addrnbr':'str'
}
address = cdutils.input_cleansing.cast_columns(address, address_schema)
address = address.drop(columns='load_timestamp_utc').copy()
address = customer_address_link.merge(address, how='inner', on='addrnbr')


In [ ]:
address

In [ ]:
customer_df

In [ ]:
customer_df = customer_df.merge(address, on='customer_id', how='left', indicator=True)


In [ ]:
customer_df

In [ ]:
wh_pers = DeltaTable(src.config.BRONZE / "wh_pers").to_pandas()

In [ ]:
wh_pers

In [ ]:
import pandas as pd
import src.config
from deltalake import DeltaTable
import cdutils.customer_dim # type: ignore
import src.raynham_report.fetch_data
import cdutils.input_cleansing # type: ignore

def filter_distinct_customers_assigned_to_n_raynham(df):
    """
    Unique customers that are assigned to branch in question
    """
    filtered_df = df[df['branchname'] == 'BCSB - NORTH RAYNHAM BRANCH'].copy()
    distinct_customers = filtered_df[['customer_id']].drop_duplicates()
    distinct_customers['eligibility'] = 'Assigned to N Raynham'
    return distinct_customers

def filter_distinct_customers_transacted_at_n_raynham(accts):
    """
    Unique accounts that have transacted at branch in question
    """
    data = src.raynham_report.fetch_data.fetch_rtxn_data()
    transacted = data['transacted'].copy()
    transacted_schema = {
        'acctnbr':'str'
    }
    transacted = cdutils.input_cleansing.cast_columns(transacted, transacted_schema)
    transacted = transacted.merge(accts, how='inner', on='acctnbr')
    distinct_customers = transacted[['customer_id']].drop_duplicates()
    distinct_customers['eligibility'] = 'Transacted at N Raynham within 90 days'

    return distinct_customers 

def filter_distinct_customer_near_branches(accts):
    """
    Unique Customers located near raynham or taunton

    Requested was within 5 mi of branch, but for simplicity, we match on primary zip codes
    """
    zip_codes = ["02767","02780"]
    accts_schema = {
        'zipcd'
    }
    filtered_df = accts[accts['primaryownerzipcd'].isin(zip_codes)]
    distinct_customers = filtered_df[['customer_id']].drop_duplicates()
    distinct_customers['eligibility'] = 'Zip Code in Raynham (02767) or Taunton (02767)'
    return distinct_customers
    

# def generate_raynham_report():
"""
Objective:

To create a person mailing list to North raynham Branch Construction customers

Params:

Assigned to N Raynham
Transacted at N Raynham within 90 days
Address within a 5 mile radius of N. Raynham, Raynham Center, Main Office
Owns Safe Deposit Box @ N. Raynham
Exclusions (do we have standard exclusions for all mailings? Including common ones below)
Customers under 18 years of age
Deceased customers
Charged off accounts


Data fields:

First name
Last name
Address
City
State
Zip
"""

# Pull in Base Customer Layer
base_customer_dim = DeltaTable(src.config.SILVER / "base_customer_dim").to_pandas()

# Pull in Active Accounts (create a clean customer_id for joining)
accts = DeltaTable(src.config.SILVER / "account").to_pandas()
accts = cdutils.customer_dim.orgify(accts, 'taxrptfororgnbr')
accts = accts.rename(columns={'customer_id':'org_id'}).copy()
accts = cdutils.customer_dim.persify(accts, 'taxrptforpersnbr')
accts['customer_id'] = accts['customer_id'].fillna(accts['org_id'])
accts = accts.drop(columns=['org_id']).copy()

# Assigned to Branch from Accts

assigned = filter_distinct_customers_assigned_to_n_raynham(accts)

# Transacted at Branch in last 90 days
transacted = filter_distinct_customers_transacted_at_n_raynham(accts)

# Address near Branch (zip codes)
nearby = filter_distinct_customer_near_branches(accts)

# Union/concat eligible criteria on cust_id
concat_df = pd.concat([assigned, transacted, nearby], ignore_index=True)
# concat_df = concat_df.drop_duplicates(subset=['customer_id'], keep='first')

# Inner join with base customer dim

# Append primary address


In [ ]:
concat_df

In [ ]:
base_customer_dim = base_customer_dim[[
    'customer_id',
    'customer_type',
    'customer_name'
]].copy()

In [ ]:
concat_df.groupby('eligibility').nunique()

In [ ]:
import pandas as pd
import src.config
from deltalake import DeltaTable
import cdutils.customer_dim # type: ignore
import src.raynham_report.fetch_data
import cdutils.input_cleansing # type: ignore

def filter_distinct_customers_assigned_to_n_raynham(df):
    """
    Unique customers that are assigned to branch in question
    """
    filtered_df = df[df['branchname'] == 'BCSB - NORTH RAYNHAM BRANCH'].copy()
    distinct_customers = filtered_df[['customer_id']].drop_duplicates()
    distinct_customers['eligibility'] = 'Assigned to N Raynham'
    return distinct_customers

def filter_distinct_customers_transacted_at_n_raynham(accts):
    """
    Unique accounts that have transacted at branch in question
    """
    data = src.raynham_report.fetch_data.fetch_rtxn_data()
    transacted = data['transacted'].copy()
    transacted_schema = {
        'acctnbr':'str'
    }
    transacted = cdutils.input_cleansing.cast_columns(transacted, transacted_schema)
    transacted = transacted.merge(accts, how='inner', on='acctnbr')
    distinct_customers = transacted[['customer_id']].drop_duplicates()
    distinct_customers['eligibility'] = 'Transacted at N Raynham within 90 days'

    return distinct_customers 

def filter_distinct_customer_near_branches(accts):
    """
    Unique Customers located near raynham or taunton

    Requested was within 5 mi of branch, but for simplicity, we match on primary zip codes
    """
    zip_codes = ["02767","02780"]
    accts_schema = {
        'zipcd'
    }
    filtered_df = accts[accts['zip']]
    

# def generate_raynham_report():
"""
Objective:

To create a person mailing list to North raynham Branch Construction customers

Params:

Assigned to N Raynham
Transacted at N Raynham within 90 days
Address within a 5 mile radius of N. Raynham, Raynham Center, Main Office
Owns Safe Deposit Box @ N. Raynham
Exclusions (do we have standard exclusions for all mailings? Including common ones below)
Customers under 18 years of age
Deceased customers
Charged off accounts


Data fields:

First name
Last name
Address
City
State
Zip
"""

# Pull in Base Customer Layer
base_customer_dim = DeltaTable(src.config.SILVER / "base_customer_dim").to_pandas()

# Pull in Active Accounts (create a clean customer_id for joining)
accts = DeltaTable(src.config.SILVER / "account").to_pandas()
accts = cdutils.customer_dim.orgify(accts, 'taxrptfororgnbr')
accts = accts.rename(columns={'customer_id':'org_id'}).copy()
accts = cdutils.customer_dim.persify(accts, 'taxrptforpersnbr')
accts['customer_id'] = accts['customer_id'].fillna(accts['org_id'])
accts = accts.drop(columns=['org_id']).copy()

# Assigned to Branch from Accts

assigned = filter_distinct_customers_assigned_to_n_raynham(accts)

# Transacted at Branch in last 90 days
transacted = filter_distinct_customers_transacted_at_n_raynham(accts)

# Address near Branch (zip codes)


# Union/concat eligible criteria on cust_id

# Inner join with base customer dim

# Append primary address


In [ ]:
accts.info()

In [ ]:
import pandas as pd
import src.config
from deltalake import DeltaTable
import cdutils.customer_dim # type: ignore
import cdutils.input_cleansing # type: ignore
import cdutils.add_effdate # type: ignore

# def generate_customer_address_link():
"""
Genereate silver linking able for address/customer

Uses customer_id as primary key
O+orgnbr
P+persnbr

Upstream dependencies:
silver address table

Bronze persaddruse + orgaddruse
"""
# Pers handling
persaddruse = DeltaTable(src.config.BRONZE / "persaddruse").to_pandas()
persaddruse = cdutils.customer_dim.persify(persaddruse, 'persnbr')
persaddruse = persaddruse.sort_values(by='effdate', ascending=False)
persaddruse_no_dupes = persaddruse.drop_duplicates(subset=['customer_id','addrusecd'], keep='first').reset_index(drop=True)
persaddruse_no_dupes = persaddruse_no_dupes[[
    'customer_id',
    'addrnbr',
    'addrusecd'
]].copy()

# Org handling
orgaddruse = DeltaTable(src.config.BRONZE / "orgaddruse").to_pandas()
orgaddruse = cdutils.customer_dim.orgify(orgaddruse, 'orgnbr')
orgaddruse = orgaddruse.sort_values(by='effdate', ascending=False)
orgaddruse_no_dupes = orgaddruse.drop_duplicates(subset=['customer_id','addrusecd'], keep='first').reset_index(drop=True)
orgaddruse_no_dupes = orgaddruse_no_dupes[[
    'customer_id',
    'addrnbr',
    'addrusecd'
]].copy()

# Concat
concat_df = pd.concat([persaddruse_no_dupes, orgaddruse_no_dupes], ignore_index=True)

# Add effdate
concat_df = cdutils.add_effdate.add_effdate(concat_df)



In [ ]:
concat_df

In [ ]:
import pandas as pd
import src.config
from deltalake import DeltaTable
import cdutils.customer_dim # type: ignore
import src.raynham_report.fetch_data
import cdutils.input_cleansing # type: ignore

def filter_distinct_customers_assigned_to_n_raynham(df):
    """
    Unique customers that are assigned to branch in question
    """
    filtered_df = df[df['branchname'] == 'BCSB - NORTH RAYNHAM BRANCH'].copy()
    distinct_customers = filtered_df[['customer_id']].drop_duplicates()
    distinct_customers['eligibility'] = 'Assigned to N Raynham'
    return distinct_customers

def filter_distinct_customers_transacted_at_n_raynham(accts):
    """
    Unique accounts that have transacted at branch in question
    """
    data = src.raynham_report.fetch_data.fetch_rtxn_data()
    transacted = data['transacted'].copy()
    transacted_schema = {
        'acctnbr':'str'
    }
    transacted = cdutils.input_cleansing.cast_columns(transacted, transacted_schema)
    transacted = transacted.merge(accts, how='inner', on='acctnbr')
    distinct_customers = transacted[['customer_id']].drop_duplicates()
    distinct_customers['eligibility'] = 'Transacted at N Raynham within 90 days'

    return distinct_customers 

# def generate_raynham_report():
"""
Objective:

To create a person mailing list to North raynham Branch Construction customers

Params:

Assigned to N Raynham
Transacted at N Raynham within 90 days
Address within a 5 mile radius of N. Raynham, Raynham Center, Main Office
Owns Safe Deposit Box @ N. Raynham
Exclusions (do we have standard exclusions for all mailings? Including common ones below)
Customers under 18 years of age
Deceased customers
Charged off accounts


Data fields:

First name
Last name
Address
City
State
Zip
"""

# Pull in Base Customer Layer
base_customer_dim = DeltaTable(src.config.SILVER / "base_customer_dim").to_pandas()

# Assigned to Branch from Accts
accts = DeltaTable(src.config.SILVER / "account").to_pandas()
accts = cdutils.customer_dim.orgify(accts, 'taxrptfororgnbr')
accts = accts.rename(columns={'customer_id':'org_id'}).copy()
accts = cdutils.customer_dim.persify(accts, 'taxrptforpersnbr')
accts['customer_id'] = accts['customer_id'].fillna(accts['org_id'])
accts = accts.drop(columns=['org_id']).copy()

assigned = filter_distinct_customers_assigned_to_n_raynham(accts)

# Transacted at Branch in last 90 days
transacted = filter_distinct_customers_transacted_at_n_raynham(accts)



# Address near Branch (zip codes)

# Union/concat eligible criteria on cust_id

# Inner join with base customer dim

# Append primary address


In [ ]:
address = DeltaTable(src.config.SILVER / "address").to_pandas()

In [ ]:
address

In [ ]:
persaddruse = DeltaTable(src.config.BRONZE / "persaddruse").to_pandas()

In [ ]:
persaddruse = cdutils.customer_dim.persify(persaddruse, 'persnbr')

In [ ]:
persaddruse

In [ ]:
orgaddruse = DeltaTable(src.config.BRONZE / "orgaddruse").to_pandas()

In [ ]:
orgaddruse = cdutils.customer_dim.orgify(orgaddruse, 'orgnbr')

In [ ]:
orgaddruse

In [ ]:
transacted

In [ ]:
transacted = transacted.merge(accts, how='inner', on='acctnbr')

In [ ]:
transacted

In [ ]:
import pandas as pd
import src.config
from deltalake import DeltaTable
import cdutils.customer_dim # type: ignore

# def generate_raynham_report():
"""
Objective:

To create a person mailing list to North raynham Branch Construction customers

Params:

Assigned to N Raynham
Transacted at N Raynham within 90 days
Address within a 5 mile radius of N. Raynham, Raynham Center, Main Office
Owns Safe Deposit Box @ N. Raynham
Exclusions (do we have standard exclusions for all mailings? Including common ones below)
Customers under 18 years of age
Deceased customers
Charged off accounts


Data fields:

First name
Last name
Address
City
State
Zip
"""

# Pull in Base Customer Layer
base_customer_dim = DeltaTable(src.config.SILVER / "base_customer_dim").to_pandas()

# Assigned to Branch from Accts
accts = DeltaTable(src.config.SILVER / "account").to_pandas()
accts = cdutils.customer_dim.orgify(accts, 'taxrptfororgnbr')
accts = accts.rename(columns={'customer_id':'org_id'}).copy()
accts = cdutils.customer_dim.persify(accts, 'taxrptforpersnbr')
accts['customer_id'] = accts['customer_id'].fillna(accts['org_id'])
accts = accts.drop(columns=['org_id']).copy()

# Transacted at Branch in last 90 days

# Address near Branch (zip codes)

# Union/concat eligible criteria on cust_id

# Inner join with base customer dim

# Append primary address


In [ ]:
accts

In [ ]:
import pandas as pd
import src.config
from deltalake import DeltaTable
import cdutils.customer_dim # type: ignore

def filter_distinct_customers_assigned_to_n_raynham(df):
    """
    
    """
    filtered_df = df[df['branchname'] == 'BCSB - NORTH RAYNHAM BRANCH'].copy()
    distinct_customers = filtered_df[['customer_id']].drop_duplicates()
    distinct_customers['eligibility'] = 'Assigned to N Raynham'
    return distinct_customers

# def generate_raynham_report():
"""
Objective:

To create a person mailing list to North raynham Branch Construction customers

Params:

Assigned to N Raynham
Transacted at N Raynham within 90 days
Address within a 5 mile radius of N. Raynham, Raynham Center, Main Office
Owns Safe Deposit Box @ N. Raynham
Exclusions (do we have standard exclusions for all mailings? Including common ones below)
Customers under 18 years of age
Deceased customers
Charged off accounts


Data fields:

First name
Last name
Address
City
State
Zip
"""

# Pull in Base Customer Layer
base_customer_dim = DeltaTable(src.config.SILVER / "base_customer_dim").to_pandas()

# Assigned to Branch from Accts
accts = DeltaTable(src.config.SILVER / "account").to_pandas()
accts = cdutils.customer_dim.orgify(accts, 'taxrptfororgnbr')
accts = accts.rename(columns={'customer_id':'org_id'}).copy()
accts = cdutils.customer_dim.persify(accts, 'taxrptforpersnbr')
accts['customer_id'] = accts['customer_id'].fillna(accts['org_id'])
accts = accts.drop(columns=['org_id']).copy()

assigned = filter_distinct_customers_assigned_to_n_raynham(accts)


# Transacted at Branch in last 90 days

# Address near Branch (zip codes)

# Union/concat eligible criteria on cust_id

# Inner join with base customer dim

# Append primary address


In [ ]:
assigned

In [ ]:
accts

In [ ]:
base_customer_dim